In [ ]:
import src.lightning as l

import torch
import torch.utils.data as data
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
import os

In [ ]:
L=10
N=1000
batch_size=50
ckpt_path  = './checkpoints/synth'
ckpt_model = './checkpoints/synth/...'

In [ ]:
model = l.LightningSeq2Seq(L, L, warmup=200, N=2, factor=.1)

In [ ]:
train = data.DataLoader(l.SynthDataset(L, N), batch_size=batch_size)
val = data.DataLoader(l.SynthDataset(L, N // 100), batch_size=batch_size)

In [ ]:
trainer = pl.Trainer(
    default_root_dir=ckpt_path,
    gpus=0,
    max_epochs=3,
    callbacks=[
        # Save the best checkpoint based on the maximum val_acc recorded. Saves only weights and not optimizer
        ModelCheckpoint(
            save_weights_only=False, save_top_k=1,
            mode="min", monitor="val_loss",
            every_n_epochs=1
        ),
        # Log learning rate every epoch
        LearningRateMonitor("step"),
        l.LogDistributions()
    ],
    fast_dev_run=False
)

In [ ]:
trainer.fit(model, train, val, ckpt_path=ckpt_model if os.path.exists(ckpt_model) else None)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir ./checkpoints/synth

In [ ]:
import torch.nn as nn

def greedy_decode(self, src, start_symbol, max_len=50, padding_idx=0, repeat_allowed=False):
    memory = self.transformer.encoder(self.src_embed(src), src_key_padding_mask=padding_mask(src, padding_idx))
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)

    for _ in range(max_len-1):
        out = self.transformer.decoder(self.trg_embed(ys), memory,
                                       tgt_mask=nn.Transformer.generate_square_subsequent_mask(ys.size(-1)),
                                       tgt_key_padding_mask=None)
        prob = self.generator(out[:, -1])
        if not repeat_allowed:
            prob[0, ys[0][-1]] = -100 # do not return the same word as the last word in the input
        
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.data[0]
        ys = torch.cat([ys, 
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return ys

In [ ]:
greedy_decode(model.model, torch.tensor([[1,3,5,7,9,2,4,6,8]]), 1, 9, repeat_allowed=True)

In [ ]:
x[:,-1].shape